<a href="https://colab.research.google.com/github/fedenemi/practice_university_of_michigan/blob/main/asign4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [ ]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs
# Output:
# [<networkx.classes.graph.Graph at 0x7f6e5aef6898>,
#  <networkx.classes.graph.Graph at 0x7f6e5aef69b0>,
#  <networkx.classes.graph.Graph at 0x7f6e5aef69e8>,
#  <networkx.classes.graph.Graph at 0x7f6e5aef6a20>,
#  <networkx.classes.graph.Graph at 0x7f6e5aef6a58>]

P1_Graphs is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:

Preferential Attachment ('PA')
Small World with low probability of rewiring ('SW_L')
Small World with high probability of rewiring ('SW_H')
Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

The graph_identification function should return a list of length 5 where each element in the list is either 'PA', 'SW_L', or 'SW_H'.

In [ ]:
def graph_identification():
    
    gs=[(nx.average_shortest_path_length(P1_Graphs[i]),nx.transitivity(P1_Graphs[i]),sorted({(v,k) for k,v in nx.degree_centrality(P1_Graphs[i]).items()})[-1:]) for i in range(len(P1_Graphs))]# Your Answer Here

    # Your Code Here
    
    return ["PA","SW_L" , "SW_H", "PA", "SW_H"]

graph_identification()

# Output: ['PA', 'SW_L', 'SW_H', 'PA', 'SW_H']

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes Department and ManagementSalary.

Department indicates the department in the company which the person belongs to, and ManagementSalary indicates whether that person is receiving a management position salary.

In [ ]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

# Output:
# Name: 
# Type: Graph
# Number of nodes: 1005
# Number of edges: 16706
# Average degree:  33.2458

Using network G, identify the people in the network with missing values for the node attribute ManagementSalary and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have ManagementSalary data, and predict a probability of the node receiving a management salary for nodes where ManagementSalary is missing.

Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


def salary_predictions():
    msal=nx.get_node_attributes(G,"ManagementSalary")      
    dep=nx.get_node_attributes(G,"Department")
    
    df=pd.concat([pd.Series(dep), pd.Series(msal)],axis=1)
    df.columns=["department","management_salary"]
    df.index=G.nodes()
    
    
    df["degree"]=list(G.degree().values())#({v for k,v in G.degree().items()})
    df["clustering"]=list(nx.clustering(G))
    df["hits"]=list(nx.hits(G)[0])
    df["hitsb"]=list(nx.hits(G)[1])
    df["betweenness"]=list(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    df["closeness"]=list(nx.closeness_centrality(G))
    df["pagerank"]=list(nx.pagerank(G, alpha=0.8))
    
    unknown=df[df["management_salary"].isnull()].drop(["management_salary"], axis = 1)
    
    df=df.dropna()
    y=df["management_salary"] #[:,None]    
    X=df.drop(["management_salary"], axis = 1)

    
    model=LogisticRegression(C=1, random_state=0).fit(X,y)
    print(np.mean(cross_val_score(model, X, y, cv=5, scoring='roc_auc')))
    
    
    # Your Code Here
    
    return pd.Series([p[1] for p in model.predict_proba(unknown)],index=unknown.index)
salary_predictions() # Your Answer Here

# Output:
# 0.91826796107
# 1       0.197784
# 2       0.604821
# 5       0.960076
# 8       0.149392
# 14      0.409170
# 18      0.220039
# 27      0.295180
# 30      0.333082
# 31      0.208648
# 34      0.118343
# ...

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable future_connections. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the Future Connection column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

# Output:
# Future Connection
# (6, 840)	0.0
# (4, 197)	0.0
# (620, 979)	0.0
# (519, 872)	0.0
# (382, 423)	0.0
# (97, 226)	1.0
# (349, 905)	0.0
# (429, 860)	0.0
# (309, 989)	0.0
# (468, 880)	0.0

Using network G and future_connections, identify the edges in future_connections with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in future_connections using networkx, train a sklearn classifier on those edges in future_connections that have Future Connection data, and predict a probability of the edge being a future connection for those edges in future_connections where Future Connection is missing.

Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

In [ ]:
def new_connections_predictions():
    
    fc = future_connections
    
    
    fc["pref"] = [score[2] for score in list(nx.preferential_attachment(G, fc.index))]
    fc["adamic"] =[ score[2] for score in list(nx.adamic_adar_index(G, fc.index))]    
    fc["resource_allocation"] =[score[2] for score in list(nx.resource_allocation_index(G, fc.index))]
    fc["jaccard_coefficient"] = [score[2] for score in list(nx.jaccard_coefficient(G, fc.index))]
    
    fc["common_neighbors"] = [len(list(nx.common_neighbors(G, u,v))) for u, v in zip(fc.index.str[0], fc.index.str[0])]
    
    
    unknown=fc[fc["Future Connection"].isnull()].drop(["Future Connection"], axis=1)
    fc=fc.dropna()
    y=fc["Future Connection"]
    X=fc.drop(["Future Connection"], axis=1)
    
    
    model=LogisticRegression(C=0.1, random_state=0).fit(X,y)
    print(np.mean(cross_val_score(model, X, y, cv=5, scoring='roc_auc')))
    # Your Code Here
    
    return pd.Series([p[1] for p in model.predict_proba(unknown)], index=unknown.index)

new_connections_predictions()

# Output:
# 0.906031970237
# (107, 348)    0.019258
# (542, 751)    0.014939
# (20, 426)     0.551234
# (50, 989)     0.015363
# (942, 986)    0.016489
# (324, 857)    0.015077
# (13, 710)     0.179457
# (19, 271)     0.100376
# (319, 878)    0.015771
# (659, 707)    0.016062